In [1]:
!pip install hyperopt

In [19]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [7]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

ERROR! Session/line number was not unique in database. History logging moved to new session 59


In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  
  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks= [tensorboard_callback]
  )
  return model 

def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

def get_cnn_v5(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape, padding='same'),
  Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3), 
  
  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(num_classes, activation='softmax')
  ])

In [10]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 7s 25ms/step - loss: 2.5421 - accuracy: 0.2747 - val_loss: 1.3694 - val_accuracy: 0.5434
Epoch 2/5
272/272 [==============================] - 6s 23ms/step - loss: 1.1401 - accuracy: 0.6232 - val_loss: 0.4159 - val_accuracy: 0.8684
Epoch 3/5
272/272 [==============================] - 6s 23ms/step - loss: 0.5413 - accuracy: 0.8250 - val_loss: 0.1706 - val_accuracy: 0.9484
Epoch 4/5
272/272 [==============================] - 6s 23ms/step - loss: 0.3293 - accuracy: 0.8962 - val_loss: 0.0673 - val_accuracy: 0.9785
Epoch 5/5
272/272 [==============================] - 6s 24ms/step - loss: 0.2374 - accuracy: 0.9268 - val_loss: 0.0515 - val_accuracy: 0.9854


0.9662131519274376

In [15]:
model_trained.evaluate(X_test, y_test)

138/138 [==============================] - 0s 3ms/step - loss: 0.1500 - accuracy: 0.9662


[0.15002648532390594, 0.9662131667137146]

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape, padding='same'),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),
    
    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax')
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  
  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0,
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model} 

In [0]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10), #od 100 do 200 z krokiem 10
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7)
}

In [44]:
best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 110.0, 'dropout_cnn_block_one': 0.4992870764055403, 'dropout_cnn_block_three': 0.37004142657419026, 'dropout_cnn_block_two': 0.46641466410259375, 'dropout_dense_block_one': 0.6631847517954997, 'dropout_dense_block_two': 0.3182767045931154}
accuracy=0.94399094581604
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.35812455865501847, 'dropout_cnn_block_three': 0.36693055291408877, 'dropout_cnn_block_two': 0.3204566029343743, 'dropout_dense_block_one': 0.5710412204027744, 'dropout_dense_block_two': 0.5573563054074068}
accuracy=0.960090696811676
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.3590488171429375, 'dropout_cnn_block_three': 0.4543998078073642, 'dropout_cnn_block_two': 0.4519099350987787, 'dropout_dense_block_one': 0.6922591571596508, 'dropout_dense_block_two': 0.33722565341467475}
accuracy=0.940816342830658
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.3427690817662588, 'dropout_cnn_block_three': 0.39069357430692103, 'dropout_cnn_block_two': 0.35079026661666